In [1]:
from pymongo import MongoClient
from elasticsearch import Elasticsearch, helpers
import requests
from IPython.display import JSON
import numpy as np
from fuzzywuzzy import fuzz
from joblib import Parallel, delayed
import pickle
from IPython.display import clear_output, display

In [2]:
scopus = list(MongoClient()["scopus_colombia"]["stage"].find({'DOI': {"$ne": np.nan}},{"_id":0,'Title':1,'Year':1, 'Source title':1,'Authors':1,"DOI":1}))
openalex = list(MongoClient()["openalexco"]["works"].find({'doi': {"$ne": None}},{"_id":0,"doi":1,"title":1,"publication_year":1,
                                                              "host_venue.display_name":1,
                                                             "authorships.author.display_name":1}))

In [3]:
JSON(openalex[0])

<IPython.core.display.JSON object>

In [4]:
len(openalex)

203863

In [5]:
#[i["doi"] for i in openalex]

In [6]:
%%time
p = []
n = []
counter = 0
size=5000
p_dois = []
#positives
for i in scopus:
    
    for j in openalex:
        if i["DOI"] ==  j['doi'].replace("https://doi.org/",""):
            p.append((i,j))
            p_dois.append(j['doi'].replace("https://doi.org/",""))
            counter+=1
            if counter%10 == 0:
                clear_output(wait=True)
                print(counter)
            continue
            
        if len(p) == size:
             break
    if len(p) == size:
         break

5000
CPU times: user 6min 34s, sys: 345 ms, total: 6min 34s
Wall time: 6min 34s


In [7]:
%%time
#negatives
n=[]
counter=0
n_dois = []
for i in scopus:
    for j in openalex:
        if i["DOI"] != j['doi'].replace("https://doi.org/",""):
            if i["DOI"] not in p_dois and j['doi'].replace("https://doi.org/","") not in p_dois:
                n.append((i,j))
                counter+=1
                if counter == size:
                    print(counter)
                    break
            if counter == size:
                break
            continue
    if counter == size:
        break


5000
CPU times: user 327 ms, sys: 0 ns, total: 327 ms
Wall time: 327 ms


In [8]:
len(p)

5000

In [9]:
len(n)

5000

In [10]:
p[0]

({'Authors': 'Jaramillo-Ayerbe F., Berrío-Muñoz J.',
  'Title': 'Ivermectin for crusted Norwegian scabies induced by use of topical steroids',
  'Year': 1998,
  'Source title': 'Archives of Dermatology',
  'DOI': '10.1001/archderm.134.2.143'},
 {'doi': 'https://doi.org/10.1001/archderm.134.2.143',
  'title': 'Ivermectin for Crusted Norwegian Scabies Induced by Use of Topical Steroids',
  'publication_year': 1998,
  'host_venue': {'display_name': 'Archives of Dermatology'},
  'authorships': [{'author': {'display_name': 'Felipe Jaramillo-Ayerbe'}},
   {'author': {'display_name': 'Joaquín Berrío-Muñoz'}}]})

In [11]:
n[0]

({'Authors': 'Flórez G.A.D., Mombello E.E., Voss S.',
  'Title': 'Novel technique for the calculation of eddy current losses and Lorentz forces in foil winding transformers',
  'Year': 2017,
  'Source title': 'International Journal of Applied Electromagnetics and Mechanics',
  'DOI': '10..3233/JAE-160144'},
 {'doi': 'https://doi.org/10.1000/riec.v1i1.1',
  'title': 'ARQUEOLOGIA, TURISMO E HISTÓRIA E O PARQUE NACIONAL SERRA DA CAPIVARA – PNSC/PI',
  'publication_year': 2018,
  'host_venue': {'display_name': 'Revista Interdisciplinar Encontro das Ciências - RIEC | ISSN: 2595-0959 |'},
  'authorships': [{'author': {'display_name': 'Michel Justamand'}},
   {'author': {'display_name': 'Pedro Paulo A. Funari'}},
   {'author': {'display_name': 'Andres Alarcon-Jimenez'}}]})

In [12]:
with open('dataset.pkl', 'wb') as f:
    pickle.dump({"p":p,"n":n}, f)